In [2]:
import pandas as pd
from pprint import pprint
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Reading Data

In [3]:
# GALAXY DATASET
def DatasetGALAXY():
    df = pd.read_csv('../../Data/galaxymorphology/dataset1_sydney.csv')
    df.head()
    df['target'].value_counts()
    mapping = {
        'elliptical': 0,
        'spiral':1
    }
    df['target'] = df['target'].map(mapping)
    x = df.sample(df.shape[0])
    df = x.iloc[:, :5]
    df['target'] = x.iloc[:,-1]
    return df
df = DatasetGALAXY()

In [4]:
#CAR DATASET
def DatasetCAR():
    df = pd.read_csv('../../Data/dataset1-car/car_dataset.csv')
    
    #     df['buying'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['maintenance'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['doors'].replace(['2', '3', '4', '5more'],[2,3,4,5], inplace = True)
    #     df['persons'].replace(['2', '4', 'more'],[2,4,5], inplace = True)
    #     df['lug_boot'].replace(['small', 'med', 'big'],[1,2,3], inplace = True)
    #     df['safety'].replace(['high', 'med', 'low'],[3,2,1], inplace = True)
    #     df['target'].replace(['unacc', 'acc', 'good', 'vgood'],[0,1,2,3], inplace = True)
    #     df= df[df.target !=3]
    #     df= df[df.target !=2]
    print(df['target'].nunique())
    df = df.sample(df.shape[0])
    return df

df = DatasetCAR()


4


In [19]:
#IRIS DATASET
def DatasetIRIS():
    df = pd.read_csv('../../Data/iris/iris.data')
    #df['target'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],[0,1,2], inplace = True)
    df = df.sample(df.shape[0])
    return df
df = DatasetIRIS()


In [22]:
#TITANIC DATASET
def DatasetTITANIC():
    df = pd.read_csv('../../Data/Titanic/train.csv')
    df.drop(columns = ['Cabin', 'Embarked', 'Ticket','Name','PassengerId'], inplace=True)
    df['Sex'].replace(['male', 'female'],[0,1], inplace = True)
    df =df.dropna()
    
    return df
df = DatasetTITANIC()

In [5]:
df

,buying,maintenance,doors,persons,lug_boot,safety,target
1694,low,low,4,more,small,high,good
719,high,med,4,4,big,high,acc
886,med,vhigh,2,more,med,med,unacc
1645,low,low,2,more,big,med,good
1153,med,med,4,more,small,med,acc
...,...,...,...,...,...,...,...
123,vhigh,high,2,4,big,low,unacc
1378,low,vhigh,5more,2,small,med,unacc
728,high,med,4,more,big,high,acc
39,vhigh,vhigh,3,4,med,low,unacc


## NAIVE BAYES

In [6]:
def create_table(df, label_column):
    table = {}

    # determine values for the label
    value_counts = df[label_column].value_counts().sort_index()
    table["class_names"] = value_counts.index.to_numpy()
    table["class_counts"] = value_counts.values
    
    # determine probabilities for the features
    for feature in df.drop(label_column, axis=1).columns:
        table[feature] = {}
        # determine counts
        counts = df.groupby(label_column)[feature].value_counts()
        df_counts = counts.unstack(label_column)
        # add one count to avoid "problem of rare values"
        if df_counts.isna().any(axis=None):
            df_counts.fillna(value=0, inplace=True)
            df_counts += 1

        # calculate probabilities
        df_probabilities = df_counts / df_counts.sum()
        for value in df_probabilities.index:
            probabilities = df_probabilities.loc[value].to_numpy()
            table[feature][value] = probabilities
    pprint(table) 
    return table

In [7]:
def predict_example(row, lookup_table):
    class_estimates = lookup_table["class_counts"]
    for feature in row.index:
        try:
            value = row[feature]
            probabilities = lookup_table[feature][value]
            class_estimates = class_estimates * probabilities

        # skip in case "value" only occurs in test set but not in train set
        # (i.e. "value" is not in "lookup_table")
        except KeyError:
            continue

    index_max_class = class_estimates.argmax()
    prediction = lookup_table["class_names"][index_max_class]
    
    return prediction

In [8]:
def get_overall_cm(df):
    classes = df['target'].nunique()
    if classes < 2:
        classes = 2
    overall_cm = []
    for i in range(classes):
        overall_cm.append( [0] * classes)
    return overall_cm

In [9]:
from sklearn.metrics import confusion_matrix

def Kfold(df,n_folds, test_size, overall_cm):
    # Iterate through all folds
    overall_predicted = []
    overall_actual = []
    for i in range(n_folds):
        print("\n\n\n\nTHIS IS FOLD:", i)
        #split test and train
        test = df.iloc[test_size * i : test_size*i+test_size]
        test_labels = test.iloc[:, -1]
        test.drop(columns = ['target'], inplace= True)

        train = df.iloc[test_size:]
        lookup_table = create_table(train, label_column="target")

        predictions = test.apply(predict_example, axis=1, args=(lookup_table,))
        for p,a in zip(predictions,test_labels):
            overall_predicted.append(p)
            overall_actual.append(a)
        # generate and append cm 
        overall_cm = overall_cm + confusion_matrix(test_labels, predictions)
    print(classification_report(overall_actual, overall_predicted))
    return overall_cm


# Run Naive Bayes

In [10]:
if __name__ == "__main__":
    # KFOLD take in n folds and the size of dataset
    n_folds = 2
    test_size = int(df.shape[0]/n_folds)

    # get overall cm returns a Confusion matrix with 0's shape depends on (classes)
    nbcm = Kfold(df,n_folds, test_size, get_overall_cm(df))
    print(f"\n\n\n Confusion Matrix\n{nbcm}")





THIS IS FOLD: 0


D:\DeveloperTools\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


{'buying': {'high': array([0.24747475, 0.02777778, 0.2640264 , 0.025     ]),
            'low': array([0.24747475, 0.66666667, 0.21122112, 0.6       ]),
            'med': array([0.3030303 , 0.27777778, 0.21452145, 0.35      ]),
            'vhigh': array([0.2020202 , 0.02777778, 0.31023102, 0.025     ])},
 'class_counts': array([194,  32, 602,  36], dtype=int64),
 'class_names': array(['acc', 'good', 'unacc', 'vgood'], dtype=object),
 'doors': {'2': array([0.19072165, 0.1875    , 0.25747508, 0.16666667]),
           '3': array([0.25773196, 0.3125    , 0.24916944, 0.22222222]),
           '4': array([0.25773196, 0.1875    , 0.25747508, 0.36111111]),
           '5more': array([0.29381443, 0.3125    , 0.2358804 , 0.25      ])},
 'lug_boot': {'big': array([0.35025381, 0.31428571, 0.27768595, 0.64102564]),
              'med': array([0.38071066, 0.37142857, 0.34049587, 0.33333333]),
              'small': array([0.26903553, 0.31428571, 0.38181818, 0.02564103])},
 'maintenance': {'high': ar

In [1]:
# Reference: https://www.sebastian-mantey.com/code-blog/coding-a-naive-bayes-classifier-from-scratch-python-p1-introduction
